In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
from datasets import load_dataset,load_from_disk
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

2022-12-03 11:02:18.891727: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-03 11:02:19.076387: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-03 11:02:19.076417: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-03 11:02:20.678564: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

RuntimeError: Failed to import transformers.data.data_collator because of the following error (look up to see its traceback):
No module named 'torch._C'

In [ ]:
# Load full dataset of 1.9 million examples
full_dataset = load_dataset("csv", data_files='train.csv')

Using custom data configuration default-81b6c2666d774de7


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1076.57it/s]


Dataset csv downloaded and prepared to /home/dataguy/.cache/huggingface/datasets/csv/default-81b6c2666d774de7/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 438.05it/s]


In [ ]:
full_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'subject', 'date'],
        num_rows: 14991
    })
})

In [ ]:
cols_to_remove = list(full_dataset['train'].features.keys())
cols_to_remove.remove("title")
cols_to_remove.remove("text")
data=full_dataset.remove_columns(cols_to_remove)
data

DatasetDict({
    train: Dataset({
        features: ['title', 'text'],
        num_rows: 14991
    })
})

In [ ]:
dataset = data['train'].train_test_split(test_size=0.1)

test_val = dataset['test'].train_test_split(test_size=0.5)
dataset['val'] = test_val['train']
dataset['test'] = test_val['test']

dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'text'],
        num_rows: 13491
    })
    test: Dataset({
        features: ['title', 'text'],
        num_rows: 750
    })
    val: Dataset({
        features: ['title', 'text'],
        num_rows: 750
    })
})

In [ ]:
dataset.save_to_disk('title_generation_dataset')

Flattening the indices: 100%|██████████| 1/1 [00:00<00:00, 40.71ba/s]


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/dataguy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Initialize T5-base tokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-base')

/home/dataguy/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# Load the processed data
dataset = load_from_disk('title_generation_dataset')
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'text'],
        num_rows: 13491
    })
    test: Dataset({
        features: ['title', 'text'],
        num_rows: 750
    })
    val: Dataset({
        features: ['title', 'text'],
        num_rows: 750
    })
})

In [30]:
MAX_SOURCE_LEN = 512
MAX_TARGET_LEN = 128

In [31]:
def preprocess_data(data):
    
    model_inputs = tokenizer(data['text'], max_length=MAX_SOURCE_LEN, padding=True, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(data['title'], max_length=MAX_TARGET_LEN, padding=True, truncation=True)

    # Replace all pad token ids in the labels by -100 to ignore padding in the loss
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]

    model_inputs['labels'] = labels["input_ids"]

    return model_inputs

In [32]:
# Apply preprocess_data() to the whole dataset
processed_dataset = dataset.map(
    preprocess_data,
    batched=True,
    remove_columns=['text', 'title'],
    desc="Running tokenizer on dataset",
)

processed_dataset

Running tokenizer on dataset:   0%|          | 0/14 [00:00<?, ?ba/s]/home/dataguy/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Running tokenizer on dataset: 100%|██████████| 1/1 [00:00<00:00,  2.06ba/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13491
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 750
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 750
    })
})

In [33]:
batch_size = 8
num_epochs = 5
learning_rate = 5.6e-5
weight_decay = 0.01
log_every = 50
eval_every = 1000
lr_scheduler_type = "linear"

In [34]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="model-t5-base",
    evaluation_strategy="steps",
    eval_steps=eval_every,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=weight_decay,
    save_steps=500,
    save_total_limit=3,
    num_train_epochs=num_epochs,
    predict_with_generate=True,
    logging_steps=log_every,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to="wandb",
    resume_from_checkpoint=True,
)

In [4]:
# Initialize T5-base model
model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')

NameError: name 'AutoModelForSeq2SeqLM' is not defined